In [10]:
# Standard Imports
import pandas as pd
import matplotlib as plt
import seaborn as sns

# Add additional imports under here

In [ ]:
# Extracting different text features from the corpus extracted from SADII of news articles in isiZulu
file = open('SADII-Ext.News.POS.2024-01-31.zu.txt', 'r', encoding='utf-8')
lines = file.read()
lines = lines.splitlines()
file.close()
    
df_corpus = pd.DataFrame(lines, columns=['isiZulu'])
df_corpus['isiZulu'] = df_corpus['isiZulu'].str.strip()

print("Total text samples from corpus: ", len(df_corpus))
print(df_corpus.head())

Total text samples from corpus:  7472
       isiZulu
0     <LINE 1>
1      "\tPUNC
2  Abantu\tN02
3  kufanele\tV
4   balinde\tV


In [31]:
# Cleaning and Tokenization
def tokenize_text(text):
    text = text.translate(str.maketrans('', '', '.,!?;"()[]{}1234567890')).lower().split('\t')
    return text

df_corpus_text = pd.DataFrame(columns=['text'])
df_corpus_tokenized = pd.DataFrame(columns=['tokens', 'pos_tag'])
text = ''
for corpus_text in df_corpus['isiZulu']:
    if '<line' in corpus_text.lower():
        if text:
            text = text.strip().replace('"', '')
            df_corpus_text.loc[len(df_corpus_text)] = text
            
        text = ''
        continue
    
    tokens = tokenize_text(corpus_text)
    text += tokens[0] + ' '
    if 'punc' in corpus_text.lower():
        continue
    
    df_corpus_tokenized.loc[len(df_corpus_tokenized)] = tokens

print("Total tokens extracted: ", len(df_corpus_tokenized))
print(df_corpus_tokenized.head())
print()
print("Total text samples: ", len(df_corpus_text))
print(df_corpus_text.head())

Total tokens extracted:  5974
              tokens pos_tag
0             abantu       n
1           kufanele       v
2            balinde       v
3       bayokwaziswa       v
4  ngokusemthethweni     adv

Total text samples:  599
                                                text
0  abantu kufanele balinde bayokwaziswa ngokusemt...
1  akusho lutho ukuthi lemali izobe isesikhwameni...
2   amabhuku ngawayisa kumuntu okwazi ukuwacubungula
3  angazi ngingathini kodwa kuyaphoxa ukuthi usha...
4                angazi ngoba eyami kade ngayikhipha
